# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

%load_ext dotenv
%dotenv

In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

c:\Users\Igor\.conda\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [9]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))

prices_dd = dd.read_parquet(files).set_index("ticker")



In [11]:
prices_dd.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.532219,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.206841,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.712807,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.276844,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.299919,2819626,Health Care,Life Sciences Tools & Services,2000
...,...,...,...,...,...,...,...,...,...,...
ZTS,2024-06-27,174.490005,181.660004,174.259995,176.460007,176.460007,4164700,Health Care,Pharmaceuticals,2024
ZTS,2024-06-28,176.460007,177.070007,173.029999,173.360001,173.360001,4189600,Health Care,Pharmaceuticals,2024
ZTS,2024-07-01,173.259995,175.460007,172.000000,172.580002,172.580002,1589800,Health Care,Pharmaceuticals,2024


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [48]:
prices_dd = prices_dd.repartition(npartitions=100)
prices_dd.npartitions

100

In [49]:
# Write your code below.
dd_feat = ( prices_dd.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1)))
    .assign( Adj_Close_Lag = lambda x: (x['Adj Close'].shift(1)))
    .assign( returns = lambda x: (x['Adj Close'] / x['Adj_Close_Lag'])-1 )
    .assign( hi_lo_range = lambda x: x['High'] - x['Low'])
     )


C:\Users\Igor\AppData\Local\Temp\ipykernel_7692\3465780647.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = ( prices_dd.groupby('ticker', group_keys=False).apply(


In [50]:
dd_feat.npartitions

100

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [60]:
prices_df=dd_feat.compute()
type(prices_df)


pandas.core.frame.DataFrame

In [61]:
prices_df

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_Lag,returns,hi_lo_range
ticker,,,,,,,,,,,,,,
ALGN,2001-01-30,18.125000,18.187500,16.250000,16.875000,16.875000,812300,Health Care,Health Care Supplies,2001,NaN,NaN,NaN,1.937500
ALGN,2001-01-31,17.125000,17.187500,13.437500,13.500000,13.500000,1212700,Health Care,Health Care Supplies,2001,16.875000,16.875000,-0.200000,3.750000
ALGN,2001-02-01,13.125000,14.375000,13.000000,14.187500,14.187500,669100,Health Care,Health Care Supplies,2001,13.500000,13.500000,0.050926,1.375000
ALGN,2001-02-02,14.000000,14.562500,13.750000,14.125000,14.125000,775000,Health Care,Health Care Supplies,2001,14.187500,14.187500,-0.004405,0.812500
ALGN,2001-02-05,14.437500,14.500000,13.875000,14.125000,14.125000,489400,Health Care,Health Care Supplies,2001,14.125000,14.125000,0.000000,0.625000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EIX,2024-06-27,71.870003,72.070000,71.529999,72.040001,72.040001,1039800,Utilities,Electric Utilities,2024,71.849998,71.849998,0.002644,0.540001
EIX,2024-06-28,72.279999,72.449997,71.239998,71.809998,71.809998,2647100,Utilities,Electric Utilities,2024,72.040001,72.040001,-0.003193,1.209999
EIX,2024-07-01,72.400002,72.400002,70.910004,71.190002,71.190002,1554600,Utilities,Electric Utilities,2024,71.809998,71.809998,-0.008634,1.489998


In [66]:
prices_df = ( prices_df.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign ( av_ret_10days = x['returns'].rolling(10).mean()))
)
prices_df

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_Lag,returns,hi_lo_range,av_ret_10days
ticker,,,,,,,,,,,,,,,
ALGN,2001-01-30,18.125000,18.187500,16.250000,16.875000,16.875000,812300,Health Care,Health Care Supplies,2001,NaN,NaN,NaN,1.937500,NaN
ALGN,2001-01-31,17.125000,17.187500,13.437500,13.500000,13.500000,1212700,Health Care,Health Care Supplies,2001,16.875000,16.875000,-0.200000,3.750000,NaN
ALGN,2001-02-01,13.125000,14.375000,13.000000,14.187500,14.187500,669100,Health Care,Health Care Supplies,2001,13.500000,13.500000,0.050926,1.375000,NaN
ALGN,2001-02-02,14.000000,14.562500,13.750000,14.125000,14.125000,775000,Health Care,Health Care Supplies,2001,14.187500,14.187500,-0.004405,0.812500,NaN
ALGN,2001-02-05,14.437500,14.500000,13.875000,14.125000,14.125000,489400,Health Care,Health Care Supplies,2001,14.125000,14.125000,0.000000,0.625000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EIX,2024-06-27,71.870003,72.070000,71.529999,72.040001,72.040001,1039800,Utilities,Electric Utilities,2024,71.849998,71.849998,0.002644,0.540001,-0.002025
EIX,2024-06-28,72.279999,72.449997,71.239998,71.809998,71.809998,2647100,Utilities,Electric Utilities,2024,72.040001,72.040001,-0.003193,1.209999,-0.002208
EIX,2024-07-01,72.400002,72.400002,70.910004,71.190002,71.190002,1554600,Utilities,Electric Utilities,2024,71.809998,71.809998,-0.008634,1.489998,-0.002745


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return? </br>
We could calculate the moving average using Dask data frame.</br></br>
+ Would it have been better to do it in Dask? Why?</br>
Dask  providing more efficient calculations on large datasets. It allows for parallel processing, which speeds up computation. Additionally, Dask is a convenient tool for storing results in Parquet files, which are efficient for both storage and retrieval.</br>

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.